In [1]:
# !python -m spacy download en_core_web_sm
# import spacy
# spacy.load('en_core_web_sm')

In [2]:
import spacy
import torch
import torchtext
from torchtext.legacy import datasets, data
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# !pip install transformers
from transformers import BertTokenizer

In [14]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [124]:
# tokenizer.tokenize("This is sparta!!!!!!")
max_input_size = tokenizer.max_model_input_sizes["bert-base-uncased"]
assert max_input_size == 512


def fun(sentence):
    return tokenizer.tokenize(sentence)[:max_input_size-2]

text_field = data.Field(
    init_token=tokenizer.cls_token,
    eos_token=tokenizer.sep_token,
    pad_token=tokenizer.pad_token,
    unk_token=tokenizer.unk_token,
    tokenize=fun,
    batch_first=True, use_vocab=False,
)
label_field = data.LabelField(dtype=torch.float)  # torch.float because GPUs use floats


In [125]:
train, test = datasets.IMDB.splits(text_field=text_field, label_field=label_field)

In [126]:
len(max(train_set, key=lambda x: len(x.text)).text)

# but we can do better!
train_buckets, valid_buckets, test_buckets = data.BucketIterator.splits(
    (train_set, valid_set, test), batch_size=128, device=device
)

In [132]:
from transformers import BertModel

bert_model = BertModel.from_pretrained("bert-base-uncased")

class NLPBert(nn.Module):
    def __init__(self, hidden_size: int, num_layers: int, dropout: float, bidirectional: bool, out_layers: int):
        super().__init__()
        self.bert = bert_model
        self.gru = nn.GRU(
           input_size=self.bert.config.hidden_size, 
           hidden_size=hidden_size,
           num_layers=num_layers,
           batch_first=True,
           dropout=0 if num_layers < 2 else dropout,
           bidirectional=bidirectional,
        )
        self.linear = nn.Linear(
            in_features=hidden_size*2 if bidirectional else hidden_size,
            out_features=out_layers,
        )
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input):
        bert_output = self.bert(input)[0]
        _, gru_h_n = self.gru(bert_output)

        if self.gru.bidirectional:
            zmienna = torch.cat((gru_h_n[-2, :, :], gru_h_n[-1, :, :]), dim=1)
            dropout = self.dropout(zmienna)
        else:
            dropout = self.dropout(gru_h_n[-1, :, :])
        return self.linear(dropout)

model = NLPBert(
    hidden_size=256,
    num_layers=2,
    dropout=0.25,
    bidirectional=True,
    out_layers=1,
)
# Stochastic gradient descent SGD
# minimalizować funkcję kosztu (szukanie minimum)

import torch.optim as optim

optimiser = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (

In [133]:
train_losses, train_metrics = train(model, train_buckets, optimiser, criterion)
# validated_losses, validated_metrics = validate(model, valid_buckets, criterion)

print()
print("Train metrics", np.mean(train_losses), np.mean(train_metrics))
# print("Validation metrics", np.mean(validated_losses), np.mean(validated_metrics))


  0%|          | 0/101 [00:01<?, ?it/s]


RuntimeError: ignored

In [128]:
ciretrion = criterion.to(device)
model = model.to(device)

def binary_accuracy(prediction, target):
    prediction = F.sigmoid(prediction)
    prediction = torch.round(prediction)
    
    compared = (prediction == target).float()
    return torch.mean(compared)


T = torch.tensor
binary_accuracy(T([0, 0.5, .2, 0.001, 0.8]), T([0, 1, 1, 1, 1]))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(1.)

In [129]:
import numpy as np
import tqdm

def train(mod, data, optimiser, criterion):
    losses = []
    metrics = []
    mod = mod.to(device)

    # train pozwala na akumulację błędów, które potem będziemy propagować wstecz
    mod.train()

    d = []
    for i, x in enumerate(data):
        if i > 100:
            break
        d.append(x)

    for bucket in tqdm.tqdm(d):
        optimiser.zero_grad()
        output = mod(bucket.text).squeeze(1)
        loss = criterion(output, bucket.label)
        metric = binary_accuracy(output, bucket.label)
        losses.append(loss.item())
        metrics.append(metric.item())
        loss.backward()
        optimiser.step()
        
        # print(np.mean(losses), losses[-1], np.mean(metrics), metrics[-1])

    return losses, metrics

In [ ]:
def validate(mod, data, criterion):
    losses = []
    metrics = []

    # wyłącza akumulacje błędów (z którego korzystaliśmy w train)
    mod.eval()

    i = 0
    for bucket in tqdm.tqdm(data):
        i += 1
        output = mod(bucket.text).squeeze(0).squeeze(1)
        output2 = F.sigmoid(output)
        # print(" ".join(text_field.vocab.itos[t] for t in bucket.text[0]))
        # print(f"{bucket.label[0]}")
        # print(f"{output2[0]}")
        # if i > 10:
        #     break
        loss = criterion(output, bucket.label)
        metric = binary_accuracy(output, bucket.label)
        losses.append(loss.item())
        metrics.append(metric.item())        
        # print(np.mean(losses), losses[-1], np.mean(metrics), metrics[-1])

    return losses, metrics



In [ ]:
tokenizer = spacy.load("en_core_web_sm")

In [ ]:
validated_losses, validated_metrics = validate(model, valid_buckets, criterion)
print("Validation metrics", np.mean(validated_losses), np.mean(validated_metrics))

In [131]:
def s2(model, sentence: str) -> bool:
    tokenized = fun(sentence)
    converted = tokenizer.convert_tokens_to_ids(tokenized)
    converted.insert(0, tokenizer.cls_token_id)
    converted.append(tokenizer.sep_token_id)
    inp = torch.LongTensor(converted).to(device)
    inp = inp.unsqueeze(0)
    model.eval()
    output = model(inp)
    m = F.sigmoid(output)
    print(sentence, m.to("cpu").detach().numpy().ravel())

s2(model, "it's bad")
s2(model, "it's good")
s2(model, "it's very bad")
s2(model, "it's very good")
s2(model, "it's aweful")
s2(model, "it's awesome")
s2(model, "it's great")
s2(model, "i like this film")
s2(model, "i don't like this film")
s2(model, "it is the best movie I have ever seen")
s2(model, "it is the worst movie I have ever seen")
s2(model, "borat was the great success, i very much liked this movie, best movie ever")
s2(model, "very boring movie I didn't like it")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


it's bad [0.56450737]
it's good [0.56762856]
it's very bad [0.55605626]
it's very good [0.56836075]
it's aweful [0.61598116]
it's awesome [0.5641318]
it's great [0.5496266]
i like this film [0.5662937]
i don't like this film [0.5527819]
it is the best movie I have ever seen [0.5624218]
it is the worst movie I have ever seen [0.5588509]
borat was the great success, i very much liked this movie, best movie ever [0.5198392]
very boring movie I didn't like it [0.5954483]


In [ ]:
def sentiment(model, sentence: str) -> bool:
    tokens = tokenizer.tokenizer(sentence)
    tokenized = [text_field.vocab.stoi[t.text] for t in tokens]
    print("|".join(text_field.vocab.itos[t] for t in tokenized))
    result = F.sigmoid(model(torch.LongTensor(tokenized).unsqueeze(1).to(device)))
    print(result.to("cpu").detach().numpy().ravel(), sentence)

sentiment(model, "it's bad")
sentiment(model, "it's good")
sentiment(model, "it's very bad")
sentiment(model, "it's very good")
sentiment(model, "it's aweful")
sentiment(model, "it's awesome")
sentiment(model, "it's great")
sentiment(model, "i like this film")
sentiment(model, "i don't like this film")
sentiment(model, "it is the best movie I have ever seen")
sentiment(model, "it is the worst movie I have ever seen")
sentiment(model, "borat was the great success, i very much liked this movie, best movie ever")
sentiment(model, "very boring movie I didn't like it")

In [ ]:
shawshank = """
Why do I want to write the 234th comment on The Shawshank Redemption? I am not sure - almost everything that could be possibly said about it has been said. But like so many other people who wrote comments, I was and am profoundly moved by this simple and eloquent depiction of hope and friendship and redemption.

The only other movie I have ever seen that effects me as strongly is To Kill a Mockingbird. Both movies leave me feeling cleaner for having watched them.
"""

sentiment(model, shawshank)

shawshank2 = """
This film is nothing but one cliche after another. Having seen many of the 100's of prison films made from the early 30's to the 50's, I was able to pull almost every minute of Shawcrap from one of those films.

While it is visually well made and acted, the story has every plot point of the "standard" prison film. They have the evil warden, innocent main character, friendly guilty guy, thugs and one good old prisoner. Don't waste your time on this one. Rent or buy some of the classic's of the prison genre
"""
sentiment(model, shawshank2)

reddit = """............ . . . . . . .. ... … ...................................................................................................."""
sentiment(model, reddit)


klasa = """
Kant has written a treatise on _The Vital Powers_; but I should like to
write a dirge on them, since their lavish use in the form of knocking,
hammering, and tumbling things about has made the whole of my life a
daily torment. Certainly there are people, nay, very many, who will
smile at this, because they are not sensitive to noise; it is precisely
these people, however, who are not sensitive to argument, thought,
poetry or art, in short, to any kind of intellectual impression: a fact
to be assigned to the coarse quality and strong texture of their brain
tissues.
"""

sentiment(model, klasa)

In [ ]:
sentiment(model, "great success")
# Funkcja kosztu, im bliżej 1 (target) tym funkcja kosztu maleje.

target = torch.ones([1, 1], dtype=torch.float32)  # 64 classes, batch size = 10
input_ = torch.full([1, 1], 0.1)  # A prediction (logit)

print(F.binary_cross_entropy_with_logits(input_, target))

target = torch.ones([1, 1], dtype=torch.float32)  # 64 classes, batch size = 10
input_ = torch.full([1, 1], 0.4)  # A prediction (logit)

print(F.binary_cross_entropy_with_logits(input_, target))

target = torch.ones([1, 1], dtype=torch.float32)  # 64 classes, batch size = 10
input_ = torch.full([1, 1], 0.7)  # A prediction (logit)

print(F.binary_cross_entropy_with_logits(input_, target))

target = torch.ones([1, 1], dtype=torch.float32)  # 64 classes, batch size = 10
input_ = torch.full([1, 1], 0.9)  # A prediction (logit)

print(F.binary_cross_entropy_with_logits(input_, target))

target, input_

In [ ]:
>>> rnn = nn.RNN(3, 2, 1)
>>> input = torch.randn(5, 3, 3)
>>> h0 = torch.randn(1, 3, 2)
>>> output, hn = rnn(input, h0)
output, hn

In [ ]:
>>> # an Embedding module containing 10 tensors of size 3
>>> embedding = nn.Embedding(100, 19)
>>> # a batch of 2 samples of 4 indices each
>>> input = torch.LongTensor([[1,98,1,0, 4,3,2,9],[4,3,2,9, 4,3,2,9]])
>>> embedding(input)